In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets('MNIST-data',one_hot=True)

#None means tensor 的第一维度可以是任意维度
#/255. 做均一化
input_x=tf.placeholder(tf.float32,[None,28*28])/255.
#输出是一个one hot的向量
output_y=tf.placeholder(tf.int32,[None,10])


Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [3]:
#输入层 [28*28*1]
input_x_images=tf.reshape(input_x,[-1,28,28,1])
#从(Test)数据集中选取3000个手写数字的图片和对应标签

test_x=mnist.test.images[:3000] #image
test_y=mnist.test.labels[:3000] #label

#隐藏层
#conv1 5*5*32
#layers.conv2d parameters
#inputs 输入，是一个张量
#filters 卷积核个数，也就是卷积层的厚度
#kernel_size 卷积核的尺寸
#strides: 扫描步长
#activation: 激活函数
conv1=tf.layers.conv2d(
    inputs=input_x_images,
    filters=32,
    kernel_size=[5,5],
    strides=1,
    padding='same',
    activation=tf.nn.relu
)
print(conv1)

#输出变成了 [28*28*32]

#pooling layer1 2*2
#tf.layers.max_pooling2d
#inputs 输入，张量必须要有四个维度
#pool_size: 过滤器的尺寸

pool1=tf.layers.max_pooling2d(
    inputs=conv1,
    pool_size=[2,2],
    strides=2
)
print(pool1)
#输出变成了[?,14,14,32]

#conv2 5*5*64
conv2=tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5,5],
    strides=1,
    padding='same',
    activation=tf.nn.relu
)

#pool2 2*2
pool2=tf.layers.max_pooling2d(
    inputs=conv2,
    pool_size=[2,2],
    strides=2
)

#flat(平坦化)
flat=tf.reshape(pool2,[-1,7*7*64])

#densely-connected layers 全连接层 1024
#tf.layers.dense
#inputs: 张量
#units： 神经元的个数
#activation: 激活函数
dense=tf.layers.dense(
    inputs=flat,
    units=1024,
    activation=tf.nn.relu
)

print(dense)

#dropout
#tf.layers.dropout
#inputs 张量
#rate 丢弃率
#training 是否是在训练的时候丢弃
dropout=tf.layers.dropout(
    inputs=dense,
    rate=0.5,
)
print(dropout)

#输出层，不用激活函数（本质就是一个全连接层）
logits=tf.layers.dense(
    inputs=dropout,
    units=10
)
print(logits)

#计算误差 cross entropy（交叉熵），再用Softmax计算百分比的概率
#tf.losses.softmax_cross_entropy
#onehot_labels: 标签值
#logits: 神经网络的输出值
loss=tf.losses.softmax_cross_entropy(onehot_labels=output_y,
                                     logits=logits)
# 用Adam 优化器来最小化误差,学习率0.001 类似梯度下降
print(loss)
train_op=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)


#精度：计算预测值和实际标签的匹配程度
#tf.metrics.accuracy
#labels：真实标签
#predictions: 预测值
#Return: (accuracy,update_op)accuracy 是一个张量准确率，update_op 是一个op可以求出精度。
#这两个都是局部变量
accuracy_op=tf.metrics.accuracy(
    labels=tf.argmax(output_y,axis=1),
    predictions=tf.argmax(logits,axis=1)
)[1] #是1是因为这里是要得到一个op

Tensor("conv2d/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("dropout/Identity:0", shape=(?, 1024), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)
Tensor("softmax_cross_entropy_loss/value:0", shape=(), dtype=float32)


In [4]:
#创建会话
sess=tf.Session()
#初始化变量
init=tf.group(tf.global_variables_initializer(),
              tf.local_variables_initializer())
sess.run(init)

for i in range(20000):
    batch=mnist.train.next_batch(50) #从Train（训练）数据集中取‘下一个’样本
    train_loss,train_op_=sess.run([loss,train_op],{input_x:batch[0],output_y:batch[1]})
    if i%100==0:
        test_accuracy=sess.run(accuracy_op,{input_x:test_x,output_y:test_y})
        print("Step=%d, Train loss=%.4f,[Test accuracy=%.2f]"%(i,train_loss,
                                                               test_accuracy))

#测试： 打印20个预测值和真实值
test_output=sess.run(logits,{input_x:test_x[:20]})
inferenced_y=np.argmax(test_output,1)
print(inferenced_y,'Inferenced numbers')#推测的数字
print(np.argmax(test_y[:20],1),'Real numbers')
sess.close()

Step=0, Train loss=2.3136,[Test accuracy=0.13]
Step=100, Train loss=2.2837,[Test accuracy=0.16]
Step=200, Train loss=2.2745,[Test accuracy=0.18]
Step=300, Train loss=2.2548,[Test accuracy=0.21]
Step=400, Train loss=2.2401,[Test accuracy=0.24]
Step=500, Train loss=2.2342,[Test accuracy=0.26]
Step=600, Train loss=2.1938,[Test accuracy=0.29]
Step=700, Train loss=2.1975,[Test accuracy=0.32]
Step=800, Train loss=2.1357,[Test accuracy=0.35]
Step=900, Train loss=2.0644,[Test accuracy=0.38]
Step=1000, Train loss=2.0004,[Test accuracy=0.40]
Step=1100, Train loss=1.8784,[Test accuracy=0.43]
Step=1200, Train loss=1.8245,[Test accuracy=0.45]
Step=1300, Train loss=1.5884,[Test accuracy=0.47]
Step=1400, Train loss=1.3994,[Test accuracy=0.48]
Step=1500, Train loss=1.2646,[Test accuracy=0.50]
Step=1600, Train loss=1.1792,[Test accuracy=0.51]
Step=1700, Train loss=0.8201,[Test accuracy=0.53]
Step=1800, Train loss=0.8542,[Test accuracy=0.54]
Step=1900, Train loss=0.6799,[Test accuracy=0.56]
Step=2000, T

Step=16300, Train loss=0.1050,[Test accuracy=0.88]
Step=16400, Train loss=0.0775,[Test accuracy=0.88]
Step=16500, Train loss=0.1257,[Test accuracy=0.88]
Step=16600, Train loss=0.2493,[Test accuracy=0.88]
Step=16700, Train loss=0.0951,[Test accuracy=0.88]
Step=16800, Train loss=0.0993,[Test accuracy=0.88]
Step=16900, Train loss=0.1461,[Test accuracy=0.88]
Step=17000, Train loss=0.1309,[Test accuracy=0.88]
Step=17100, Train loss=0.2228,[Test accuracy=0.88]
Step=17200, Train loss=0.0754,[Test accuracy=0.88]
Step=17300, Train loss=0.0626,[Test accuracy=0.88]
Step=17400, Train loss=0.0804,[Test accuracy=0.88]
Step=17500, Train loss=0.0897,[Test accuracy=0.88]
Step=17600, Train loss=0.0626,[Test accuracy=0.88]
Step=17700, Train loss=0.0463,[Test accuracy=0.88]
Step=17800, Train loss=0.1500,[Test accuracy=0.88]
Step=17900, Train loss=0.1457,[Test accuracy=0.88]
Step=18000, Train loss=0.2158,[Test accuracy=0.88]
Step=18100, Train loss=0.0937,[Test accuracy=0.88]
Step=18200, Train loss=0.1410,[